# Comparison of Four Methods for Solving the Same PDE

In this notebook, we explore and compare four different numerical methods for solving the same partial differential equation (PDE):

$$
\varepsilon \cdot (-\Delta u) + \nabla u \cdot w + \partial_t u = ((\varepsilon \pi^2 - 1) \sin(\pi x) + \pi \cos(\pi x)) \cdot e^{-t}
$$

The domain is defined as:

- Spatial domain: $\Omega = [0, 1]$
- Time interval: $t \in [0, 1]$

## Analytical Solution

The analytical solution to this PDE is:

$$
u(x, t) = \sin(\pi x) \cdot e^{-t}
$$

## Initial and Boundary Conditions

- **Initial condition**: $u(x, 0) = \sin(\pi x)$
- **Boundary conditions**: Dirichlet boundary conditions with $u(0, t) = u(1, t) = 0$

## Methods Compared

We investigate the performance of the following four methods:

1. **Continuous Galerkin (CG)**  
   This standard method performs poorly for the given problem due to stability issues.

2. **SUPG (as described in the literature)**  
   A stabilized version of CG that adds artificial diffusion aligned with the flow direction.

3. **SUPG (Eigen implementation)**  
   Our own implementation of SUPG using the Eigen library, offering improved numerical stability.

4. **Discontinuous Galerkin (DG)**  
   A fully discontinuous approach that handles convection-dominated problems well, at the cost of increased degrees of freedom.

Each method is tested under the same conditions, and their accuracy and stability are analyzed in comparison with the analytical solution.


In [17]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
from ngsolve.webgui import Draw
from netgen.occ import *

T = 1 # length of the time interval
shape = Rectangle(1,T).Face()
shape.edges.Min(X).name="left"
shape.edges.Max(X).name="right"
shape.edges.Min(Y).name="bottom"
shape.edges.Max(Y).name="top"
mesh = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=0.05))
Draw(mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

# Classical CG weakformulation performs bad in the convection-dominated regime

In [18]:
fes = H1(mesh, order=1, dirichlet="bottom|right|left")
u,v = fes.TnT()
n = specialcf.normal(2)
w = 1 # the wind speed
eps = 0.001 # the diffusion coefficient

ux = CoefficientFunction((grad(u)[0]))
B = CoefficientFunction((w,1))
a = BilinearForm(fes, symmetric=False)
a += eps*ux*grad(v)[0]*dx 
a += (B*grad(u))*v*dx
a.Assemble()

f1 = CoefficientFunction(((eps*pi**2-1)*sin(pi*x)+pi*cos(pi*x))*exp(-y))
f = LinearForm(fes)
f += f1*v*dx
f.Assemble()

gfu = GridFunction(fes)
gfu.Set(sin(pi*x),BND)  # initial condition

res = f.vec.CreateVector()
res.data = f.vec - a.mat * gfu.vec
gfu.vec.data += a.mat.Inverse(fes.FreeDofs()) * res
Draw(gfu)

uexact = sin(pi*x)*exp(-y)
l2error = sqrt(Integrate((gfu-(uexact))**2, mesh))
print(l2error)
Draw(uexact, mesh, "exact")

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

0.0006704186369805753


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

# Notebook for the SUPG formulation from the Paper

In [19]:
order =3
fes = H1(mesh, order=order, dirichlet="bottom|right|left")
u,v = fes.TnT()
n = specialcf.normal(2)
w = 1 # the wind speed
eps = 0.01 # the diffusion coefficient

h = specialcf.mesh_size
jac = specialcf.JacobianMatrix(mesh.dim)
M = CF( [[2/sqrt(3), 1/sqrt(3)], [1/sqrt(3), 2/sqrt(3)]] )
expr = jac.trans * M * jac
Cinv = 36*order**2
ws = CF((w,1))
tau =  InnerProduct(ws,expr *ws)
gamma_T = 1/sqrt(tau + (Cinv*eps/h**2)**2)

def gradx(u):
    return grad(u)[0]
def dt(u):
    return grad(u)[1]

uxx = u.Operator("hesse")[0,0]

a = BilinearForm(fes, symmetric=False)
a += eps*gradx(u)*gradx(v)*dx + (ws*grad(u))*v*dx
a += gamma_T * (-eps*uxx + ws * grad(u))*(ws* grad(v)) * dx
a.Assemble()

f1 = CoefficientFunction(((eps*pi**2-1)*sin(pi*x)+pi*cos(pi*x))*exp(-y))
f = LinearForm(fes)
f += f1*v*dx
f.Assemble()

gfu = GridFunction(fes)
gfu.Set(sin(pi*x),BND)  # initial condition

res = f.vec.CreateVector()
res.data = f.vec - a.mat * gfu.vec
gfu.vec.data += a.mat.Inverse(fes.FreeDofs()) * res
Draw(gfu)

uexact = sin(pi*x)*exp(-y)
l2error = sqrt(Integrate((gfu-(uexact))**2, mesh))
print(l2error)
Draw(uexact, mesh, "exact")

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

0.0017149238562714603


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

# Notebook for my own SUPG-formulation

In [20]:
order =3
fes = H1(mesh, order=order, dirichlet="bottom|right|left")
u,v = fes.TnT()
n = specialcf.normal(2)
w = 1 # the wind speed
eps = 0.01 # the diffusion coefficient

gamma_0 = 1
Cinv = order**4

h = specialcf.mesh_size
gamma_T = gamma_0 * h / sqrt(w**2 + Cinv * eps)

def gradx(u):
    return grad(u)[0]
def dt(u):
    return grad(u)[1]

uxx = u.Operator("hesse")[0,0]

a = BilinearForm(fes, symmetric=False)
a += eps*gradx(u)*gradx(v)*dx + (ws*grad(u))*v*dx
a += gamma_T * (-eps*uxx + ws * grad(u))*(ws* grad(v)) * dx
a.Assemble()

f1 = CoefficientFunction(((eps*pi**2-1)*sin(pi*x)+pi*cos(pi*x))*exp(-y))
f = LinearForm(fes)
f += f1*v*dx
f.Assemble()

gfu = GridFunction(fes)
gfu.Set(sin(pi*x),BND)  # initial condition

res = f.vec.CreateVector()
res.data = f.vec - a.mat * gfu.vec
gfu.vec.data += a.mat.Inverse(fes.FreeDofs()) * res
Draw(gfu)

uexact = sin(pi*x)*exp(-y)
l2error = sqrt(Integrate((gfu-(uexact))**2, mesh))
print(l2error)
Draw(uexact, mesh, "exact")

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

0.07468539948840842


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

# Notebook for DG convection-dominated

In [21]:
order=3
fes = L2(mesh, order=order, dgjumps=True)
u,v = fes.TnT()
w = 1
eps = 0.01
lam = 2
h = specialcf.mesh_size

dS = dx(skeleton=True) 
jump_u = u-u.Other()
jump_v = v-v.Other()
n = specialcf.normal(2)
avgu = 0.5*n[0] * (grad(u)[0]+grad(u.Other())[0])
avgv= 0.5*n[0] * (grad(v)[0]+grad(v.Other())[0])

ux = CoefficientFunction((grad(u)[0]))
B = CoefficientFunction((w,1))

diff = eps*ux*grad(v)[0]*dx # diffusion term
diff += -avgu * jump_v *dS #term from the partial integration

diff += -(n[0]*grad(u)[0]*v)* ds(skeleton=True) #term from the partial integration
diff += lam*order**2/h*u*v * ds(skeleton=True) # penalty term
diff += lam*order**2/h*jump_u*jump_v*dS # penalty term

uhat = IfPos(B*n, u, u.Other())
con = -B*grad(v)*u*dx
con += (B*n)*uhat *jump_v*dS
diffcon = BilinearForm(diff + con).Assemble()

f1 = CoefficientFunction(((eps*pi**2-1)*sin(pi*x)+pi*cos(pi*x))*exp(-y))
f = LinearForm(fes)
f+= f1*v*dx
f += lam*order**2/h*sin(pi*x)*v*ds(skeleton=True, definedon=mesh.Boundaries("bottom"))#weakly impose boundary condition
f.Assemble()

gfu = GridFunction(fes)
gfu.vec.data += diffcon.mat.Inverse(fes.FreeDofs()) * f.vec
Draw(gfu)

uexact = sin(pi*x)*exp(-y)
l2error = sqrt(Integrate((gfu-(uexact))**2, mesh))
print(l2error)
Draw(uexact, mesh, "exact")


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

0.04613900988051478


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene